In [121]:
import numpy as np
from numpy.linalg import norm
import pandas as pd
import itertools

## Define arm model(look at writeup for details)

$\mathrm{cos}(\theta_2 + \mathrm{acos} \frac{\mathbf{R} \cdot (1, 0)}{|\mathbf{R}|}) = \frac{{|\mathbf{R}|}^2 + x^2 - y^2}{2x\mathbf{R}}$

$\theta_2 =  \mathrm{acos} \frac{{|\mathbf{R}|}^2 + x^2 - y^2}{2x\mathbf{R}} - \mathrm{acos} \frac{\mathbf{R} \cdot (1, 0)}{|\mathbf{R}|}$

$\angle OQR = \mathrm{acos} \frac{x^2 + y^2 - {|\mathbf{R}|}^2}{2xy}$

In [122]:
def calculate_angles(P, alpha, x, y, theta_1):
    R = P + np.array([alpha * np.cos(np.radians(180) - theta_1), alpha * np.sin(np.radians(180) - theta_1)])
    if(max(x, y, norm(R)) > sum([x, y, norm(R)]) - max(x, y, norm(R))):
        return None, None
    if(abs((norm(R) ** 2 + x ** 2 - y ** 2) / (2 * x * norm(R))) > 1):
        print("invalid theta for", x, y, norm(R))
    theta_2 = np.arccos((norm(R) ** 2 + x ** 2 - y ** 2) / (2 * x * norm(R))) - (np.arccos(R[0] / norm(R)))
    angle_oqr = np.arccos((x ** 2 + y ** 2  - norm(R) ** 2)/(2 * x * y))
    return theta_2, angle_oqr

## Define measured locations

In [220]:
P = np.array([25.5, 9])

## 1. Gridsearch $x, y, \alpha$

### Define length bounds
$x_{min} <= x <= x_{max}$

$y_{min} <= y <= y_{max}$

$-\theta_{limit} <= \theta_1 <= \theta_{limit}$

In [262]:
xmin = 3
xmax = 14
ymin = 3
ymax = 14
alpha_min = 30
alpha_max = 40
theta_limit = np.deg2rad(6)

In [263]:
x_grid = list(np.linspace(xmin, xmax, 20))  # too lazy to vectorize the simulation
y_grid = list(np.linspace(ymin, ymax, 20))
alpha_grid = list(np.linspace(alpha_min, alpha_max, 10))
combinations = list(itertools.product(x_grid, y_grid, alpha_grid))
len(combinations)

4000

### Perform grid search

In [264]:
result_df = pd.DataFrame({"x": [i[0] for i in combinations], "y": [i[1] for i in combinations], "alpha": [i[2] for i in combinations]})
# below are result columns
result_df.head()

,x,y,alpha
0,3.0,3.0,30.000000
1,3.0,3.0,31.111111
2,3.0,3.0,32.222222
3,3.0,3.0,33.333333
4,3.0,3.0,34.444444


In [265]:
%%time
theta2_mins = []
theta2_maxs = []
oqr_mins = []
oqr_maxs = []
dtheta_maxs = []
dtheta_sums = []
theta2_at_theta1_0 = []
oqr_at_theta1_0 = []
for index, row in result_df.iterrows():
    x = row["x"]
    y = row["y"]
    alpha = row["alpha"]
    theta2_min = float("inf")
    theta2_max = float("-inf")
    oqr_min = float("inf")
    oqr_max = float("-inf")
    dtheta = []
    theta2_prev = 0
    thetas = list(np.linspace(-theta_limit, theta_limit))
    for index, theta_1 in enumerate(thetas):  # almost same performance as np.vectorize
        theta2, oqr = calculate_angles(P, alpha, x, y, theta_1)
        if not theta2 or not oqr:
            theta2_min = np.nan
            theta2_max = np.nan
            oqr_min = np.nan
            oqr_max = np.nan
            dtheta_sum = np.nan
            break
        theta2_min = min(theta2_min, theta2)
        theta2_max = max(theta2_max, theta2)
        oqr_min = min(oqr_min, oqr)
        oqr_max = max(oqr_max, oqr)
        if index > 0:
            dtheta.append((theta2 - theta2_prev) / (theta_1 - thetas[index - 1]))
        theta2_prev = theta2
    
    if np.isnan(theta2_min):
        dtheta_maxs.append(np.nan)
        dtheta_sums.append(np.nan)
        theta2_mins.append(theta2_min)
        theta2_maxs.append(theta2_max)
        oqr_mins.append(oqr_min)
        oqr_maxs.append(oqr_max)
        theta2_at_theta1_0.append(np.nan)
        oqr_at_theta1_0.append(np.nan)
    else:
        dtheta_maxs.append(np.rad2deg(max(dtheta)))
        dtheta_sums.append(sum(dtheta))
        theta2_mins.append(np.rad2deg(theta2_min))
        theta2_maxs.append(np.rad2deg(theta2_max))
        oqr_mins.append(np.rad2deg(oqr_min))
        oqr_maxs.append(np.rad2deg(oqr_max))
        t0, o0  = calculate_angles(P, alpha, x, y, 0)
        theta2_at_theta1_0.append(np.rad2deg(t0))
        oqr_at_theta1_0.append(np.rad2deg(o0))
        

        
result_df["theta2_min"] = theta2_mins
result_df["theta2_max"] = theta2_maxs
result_df["theta2@0"] = theta2_at_theta1_0
result_df["oqr@0"] = oqr_at_theta1_0
result_df["oqr_min"] = oqr_mins
result_df["oqr_max"] = oqr_maxs
result_df["dtheta_max"] = dtheta_maxs
result_df["dtheta_sum"] = dtheta_sums
result_df = result_df.copy()

CPU times: user 6.46 s, sys: 15.6 ms, total: 6.47 s
Wall time: 6.45 s


In [266]:
result_df = result_df.loc[np.logical_not(np.isnan(result_df["theta2_min"]))]
result_df

,x,y,alpha,theta2_min,theta2_max,theta2@0,oqr@0,oqr_min,oqr_max,dtheta_max,dtheta_sum
120,3.0,9.947368,30.000000,-99.570333,20.681775,-37.352427,83.554399,23.427666,166.881990,-416.192850,-491.029441
131,3.0,10.526316,31.111111,-99.549985,12.131132,-41.604656,83.345598,25.166319,161.542887,-398.294083,-456.031227
142,3.0,11.105263,32.222222,-103.955860,0.835439,-46.892528,84.713990,29.899904,162.336589,-378.150690,-427.897805
153,3.0,11.684211,33.333333,-112.924749,-11.252761,-52.999447,87.416788,36.211346,169.229197,-355.483448,-415.160616
163,3.0,12.263158,33.333333,-87.703285,13.678140,-41.816559,76.622282,16.827809,138.380920,-372.320982,-413.974154
...,...,...,...,...,...,...,...,...,...,...,...
3995,14.0,14.000000,35.555556,-85.366692,-72.870117,-76.984278,57.627400,47.098442,70.158254,104.867228,51.027679
3996,14.0,14.000000,36.666667,-90.423515,-77.588799,-81.943781,61.623116,51.308397,74.148520,103.961764,52.408425
3997,14.0,14.000000,37.777778,-95.153404,-82.205056,-86.692140,65.869472,55.721435,78.429872,102.111915,52.872419
3998,14.0,14.000000,38.888889,-99.631504,-86.747198,-91.272368,70.362660,60.329800,83.010302,99.678528,52.610917


### condition 1: min, max angle of $\theta_2$

$-60° <= \theta_2 <= 60°$

-55 is the lowest the lower arm can go before hitting the backplate. 50 seems like a reasonable max angle

In [272]:
cond1 = result_df.loc[np.logical_and(result_df["theta2_min"] >= -55, result_df["theta2_max"] <= 50)]
cond1

,x,y,alpha,theta2_min,theta2_max,theta2@0,oqr@0,oqr_min,oqr_max,dtheta_max,dtheta_sum
540,4.157895,11.105263,30.000000,-53.731706,24.155275,-23.284219,64.769378,18.787122,106.004563,-283.146259,-318.038509
740,4.736842,11.105263,30.000000,-51.931799,7.187861,-26.781910,64.968855,28.362695,101.131926,-235.193659,-241.405278
750,4.736842,11.684211,30.000000,-44.865575,25.335458,-18.828303,58.577815,17.147148,93.687125,-241.389892,-286.654219
761,4.736842,12.263158,31.111111,-47.998168,17.146928,-23.089077,58.710489,18.486796,93.399835,-232.063913,-266.009143
772,4.736842,12.842105,32.222222,-51.612704,6.831116,-27.949592,59.816002,22.012395,93.987289,-221.583936,-238.645598
...,...,...,...,...,...,...,...,...,...,...,...
3590,12.842105,14.000000,30.000000,-43.729711,-42.262230,-42.319186,43.767787,31.149962,57.189397,26.506944,-1.938466
3591,12.842105,14.000000,31.111111,-52.358234,-49.127212,-49.329518,46.298164,33.950200,59.738799,52.115694,11.596562
3780,13.421053,13.421053,30.000000,-52.242562,-48.167778,-48.581343,44.032584,31.530381,57.384776,60.726866,15.885763
3790,13.421053,14.000000,30.000000,-47.465063,-44.898313,-44.991860,42.991139,30.755176,56.016738,45.299013,8.452624


### condition2 : min, max angle of OQR

In [273]:
cond2 = cond1.loc[np.logical_and(result_df["oqr_min"] >= 43, result_df["oqr_max"] <= 160)]
cond2

,x,y,alpha,theta2_min,theta2_max,theta2@0,oqr@0,oqr_min,oqr_max,dtheta_max,dtheta_sum
1530,7.052632,10.526316,30.0,-54.949240,-32.085390,-43.012671,66.462928,43.692622,92.135992,-103.420772,-93.360721
1730,7.631579,10.526316,30.0,-54.906749,-36.798463,-45.047432,65.041531,43.853191,88.941697,-73.629319,-73.942166


## optimal values:
$\alpha = 30$

$x = 7.05$

$y = 10.5$